# Demonstration of the CNN used for time series classification

## Data preparation

In order to feed data to our model, we need to turn it into a keras sequence. This way it is fed to the network as batches of 1 sample each time, and there is no issue with the fact the time series have a different size. The whole preparation is done in the data_preparation function. The data are also shuffled.

In [5]:
from cnn import Turtlebot_CNN,data_preparation

train,test=data_preparation(path='auto_sending_goals/',training=0.7)#The training parameters determines how much of the dataset is dedicated to training (1=100%)


[INFO]  Data folder auto_sending_goals/11/auto_sending_goals_2021-06-18-14-25-41 already exists. Not creating.
[INFO]  Data folder auto_sending_goals/18/auto_sending_goals_2021-06-18-14-32-05 already exists. Not creating.
[INFO]  Data folder auto_sending_goals/20/auto_sending_goals_2021-06-18-14-32-54 already exists. Not creating.
[INFO]  Data folder auto_sending_goals/17/auto_sending_goals_2021-06-18-14-30-29 already exists. Not creating.
[INFO]  Data folder auto_sending_goals/9/auto_sending_goals_2021-06-18-14-17-32 already exists. Not creating.
[INFO]  Data folder auto_sending_goals/2/auto_sending_goals_2021-06-18-14-01-15 already exists. Not creating.
[INFO]  Data folder auto_sending_goals/6/auto_sending_goals_2021-06-18-14-10-33 already exists. Not creating.
[INFO]  Data folder auto_sending_goals/3/auto_sending_goals_2021-06-18-14-03-35 already exists. Not creating.
[INFO]  Data folder auto_sending_goals/15/auto_sending_goals_2021-06-18-14-27-39 already exists. Not creating.
[INFO

## Training the Model

We can now train the model using the fit method.

In [39]:
import tensorflow as tf
model=Turtlebot_CNN()
model.compile(optimizer='adam',loss='mse')
model.fit(train,epochs=1000000,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss',patience=3)])
model.summary()

- loss: 6.9849e-12
Epoch 2637/1000000
14/14 [==============================] - 0s 32ms/step - loss: 6.9800e-12
Epoch 2638/1000000
14/14 [==============================] - 0s 15ms/step - loss: 6.9767e-12
Epoch 2639/1000000
14/14 [==============================] - 0s 22ms/step - loss: 6.9737e-12
Epoch 2640/1000000
14/14 [==============================] - 0s 20ms/step - loss: 6.9701e-12
Epoch 2641/1000000
14/14 [==============================] - 0s 19ms/step - loss: 6.9678e-12
Epoch 2642/1000000
14/14 [==============================] - 0s 19ms/step - loss: 6.9640e-12
Epoch 2643/1000000
14/14 [==============================] - 0s 33ms/step - loss: 6.9611e-12
Epoch 2644/1000000
14/14 [==============================] - 0s 26ms/step - loss: 6.9413e-12
Epoch 2645/1000000
14/14 [==============================] - 0s 28ms/step - loss: 6.9380e-12
Epoch 2646/1000000
14/14 [==============================] - 0s 22ms/step - loss: 6.9334e-12
Epoch 2647/1000000
14/14 [==============================] - 0

## Testing the model

Here is a Visualization of what the CNN does 

![Visualization of the CNN](img/Schema_CNN.png)

In [43]:
import numpy as np
def results(model,test):
    res=model.predict(test)
    c=0
    for i,pred in enumerate(res):
        y=test[i][1][0]
        if np.argmax(y)==np.argmax(pred):
            c+=1
    return(c/len(res))
            

In [42]:
res

array([[9.99999642e-01, 3.70076378e-07],
       [7.16679892e-07, 9.99999285e-01],
       [1.89345914e-14, 1.00000000e+00],
       [6.73215330e-01, 3.26784700e-01],
       [9.99881983e-01, 1.17997144e-04],
       [3.05945590e-14, 1.00000000e+00]], dtype=float32)